In [1]:
import numpy as np
import pandas as pd

train_df = pd.read_csv('titanic/train.csv')
test_df = pd.read_csv('titanic/test.csv')
df = pd.concat([train_df,test_df],ignore_index=True)
df.head()

/anaconda3/envs/Python36/lib/python3.6/site-packages/ipykernel_launcher.py:6: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


,Age,Cabin,Embarked,Fare,Name,Parch,PassengerId,Pclass,Sex,SibSp,Survived,Ticket
0,22.0,NaN,S,7.2500,"Braund, Mr. Owen Harris",0,1,3,male,1,0.0,A/5 21171
1,38.0,C85,C,71.2833,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,2,1,female,1,1.0,PC 17599
2,26.0,NaN,S,7.9250,"Heikkinen, Miss. Laina",0,3,3,female,0,1.0,STON/O2. 3101282
3,35.0,C123,S,53.1000,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,4,1,female,1,1.0,113803
4,35.0,NaN,S,8.0500,"Allen, Mr. William Henry",0,5,3,male,0,0.0,373450


In [2]:
train_df.info()
print('_'*40)
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.6+ KB
________________________________________
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
PassengerId    418 non-null int64
Pclass         418 non-null int64
Name           418 non-null object
Sex            418 non-null object
Age            332 non-null float64
SibSp          418 non-null int64
Parch          418 non-null int64
Ticket         418 non-null

我们使用的字段有下面几个：

#Survived：是否生还
#Pclass：船舱等级
#Sex：性别
#Age：年龄
#SibSp：手足和配偶在船上的数量
#Parch：双亲和手足在船上的数量
#Fare：费用
#Embarked：登船港口
#passengerId:乘客身份


In [3]:
df.drop(['Name', 'Ticket', 'Cabin'], axis=1, inplace=True)
df.head()

,Age,Embarked,Fare,Parch,PassengerId,Pclass,Sex,SibSp,Survived
0,22.0,S,7.2500,0,1,3,male,1,0.0
1,38.0,C,71.2833,0,2,1,female,1,1.0
2,26.0,S,7.9250,0,3,3,female,0,1.0
3,35.0,S,53.1000,0,4,1,female,1,1.0
4,35.0,S,8.0500,0,5,3,male,0,0.0


In [4]:
df.isnull().sum()

Age            263
Embarked         2
Fare             1
Parch            0
PassengerId      0
Pclass           0
Sex              0
SibSp            0
Survived       418
dtype: int64

In [5]:
age_mean = df['Age'].mean()
df['Age'] = df['Age'].fillna(age_mean)
df['Embarked'] = df['Embarked'].fillna('S')

In [6]:
df['Sex']= df['Sex'].map({'female':0, 'male':1})

In [7]:
labels = df['Embarked'].unique().tolist()
df['Embarked'] = df['Embarked'].apply(lambda n: labels.index(n))

In [11]:
X = df.iloc[:,0:8]
Y = df.iloc[:,8]
X.head()


,Age,Embarked,Fare,Parch,PassengerId,Pclass,Sex,SibSp
0,22.0,0,7.2500,0,1,3,1,1
1,38.0,1,71.2833,0,2,1,0,1
2,26.0,0,7.9250,0,3,3,0,0
3,35.0,0,53.1000,0,4,1,0,1
4,35.0,0,8.0500,0,5,3,1,0


In [12]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=33)

In [13]:
from sklearn import preprocessing
scale = preprocessing.MinMaxScaler()
X_train = scale.fit_transform(X_train)
X_test = scale.transform(X_test)

/anaconda3/envs/Python36/lib/python3.6/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


## MLP建模
模型结构为：

input layer，8个神经元（对应8个字段）

hidden layer1，40个神经元

hidden layer2，30个神经元

output layer，1个神经元

In [16]:
from keras.models import Sequential
from keras.layers import Dense,Dropout

model = Sequential()

model.add(Dense(units=40, input_dim=8, 
                kernel_initializer='uniform', 
                activation='relu'))

model.add(Dense(units=30, 
                kernel_initializer='uniform', 
                activation='relu'))

model.add(Dense(units=1, 
                kernel_initializer='uniform',
                activation='sigmoid'))

In [17]:
model.compile(loss='binary_crossentropy', 
              optimizer='adam', metrics=['accuracy'])

train_history =model.fit(x=X_train, 
                         y=y_train, 
                         validation_split=0.1, 
                         epochs=30, 
                         batch_size=30,verbose=2)

Instructions for updating:
Use tf.cast instead.
Train on 824 samples, validate on 92 samples
Epoch 1/30
 - 1s - loss: nan - acc: 0.0218 - val_loss: nan - val_acc: 0.0000e+00
Epoch 2/30
 - 0s - loss: nan - acc: 0.0000e+00 - val_loss: nan - val_acc: 0.0000e+00
Epoch 3/30
 - 0s - loss: nan - acc: 0.0000e+00 - val_loss: nan - val_acc: 0.0000e+00
Epoch 4/30
 - 0s - loss: nan - acc: 0.0000e+00 - val_loss: nan - val_acc: 0.0000e+00
Epoch 5/30
 - 0s - loss: nan - acc: 0.0000e+00 - val_loss: nan - val_acc: 0.0000e+00
Epoch 6/30
 - 0s - loss: nan - acc: 0.0000e+00 - val_loss: nan - val_acc: 0.0000e+00
Epoch 7/30
 - 0s - loss: nan - acc: 0.0000e+00 - val_loss: nan - val_acc: 0.0000e+00
Epoch 8/30
 - 0s - loss: nan - acc: 0.0000e+00 - val_loss: nan - val_acc: 0.0000e+00
Epoch 9/30
 - 0s - loss: nan - acc: 0.0000e+00 - val_loss: nan - val_acc: 0.0000e+00
Epoch 10/30
 - 0s - loss: nan - acc: 0.0000e+00 - val_loss: nan - val_acc: 0.0000e+00
Epoch 11/30
 - 0s - loss: nan - acc: 0.0000e+00 - val_loss: 